In [1]:
import os
import pandas as pd
import cv2
from detector import get_info
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support
import seaborn as sns
import matplotlib
%matplotlib inline

Using cache found in C:\Users\virgi/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-6-12 Python-3.10.9 torch-2.0.0+cu118 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [2]:
marks_names = ['hand_mark_'+str(i).zfill(2)+axis for i in range(21) for axis in ['x','y','z']]
train_labels = marks_names + ['hand_id']

users = pd.read_csv('users.csv')
train_ids = users[users['train']==1]['user_id'].values

f_name= 'results/hand_landmarks.csv'
data = pd.read_csv(f_name)
data['hand_id'] = np.where(data['hand']=='Left', 1, 0)
data.head()

,user,sex,sign,frame,hand,...,hand_mark_19z,hand_mark_20x,hand_mark_20y,hand_mark_20z,hand_id
0,user01,0,5,4,Right,...,-0.084381,1.000000,0.348559,-0.089569,0
1,user01,0,5,2,Right,...,-0.070153,1.000000,0.274275,-0.073769,0
2,user01,0,1,24,Right,...,-0.050475,0.470683,0.976509,-0.042765,0
3,user01,0,5,3,Right,...,-0.079948,1.000000,0.341380,-0.085815,0
4,user01,0,1,23,Right,...,-0.052067,0.495761,0.972848,-0.044382,0


In [3]:
def get_data(labels):  

    train_data = data[data['user'].isin(train_ids)]
    test_data = data[~data['user'].isin(train_ids)]

    X_train = train_data[labels].values
    X_test = test_data[labels].values

    y_train = train_data['sign'].values -1
    y_test = test_data['sign'].values -1
    
    return X_train, X_test, y_train, y_test


In [4]:
model_data = pd.read_csv('results/classifier_results.csv')
#model_data.sort_values(by='val_accuracy')

In [5]:
model_data['precision'] = 0
model_data['recall'] = 0
model_data['val_precision'] = 0
model_data['val_recall'] = 0

In [6]:
for index, row in model_data.iterrows():
    print(row['name'])
    
    model_folder = 'results/models/'+row['name']
    model_r = tf.keras.models.load_model(model_folder)
    
    if row['name'].endswith('with_hand'):
        labels = marks_names + ['hand_id']
    else:
        labels = marks_names
                
    
    X_train, X_test, y_train, y_test = get_data(labels)

    pred_train = model_r.predict(X_train, verbose= False)
    y_pred = pred_train.argmax(axis=1)
    report = classification_report(y_train, y_pred, output_dict=True, labels= range(7))
    model_data.at[index, 'accuracy'] = report['accuracy']
    precision, recall, _ ,_ = precision_recall_fscore_support(y_train, y_pred)
    model_data.at[index,'precision'] = report['macro avg']['precision']
    model_data.at[index,'recall'] = report['macro avg']['recall']
    
    pred_test = model_r.predict(X_test, verbose= False)
    y_pred = pred_test.argmax(axis=1)
    report = classification_report(y_test, y_pred, output_dict=True, labels= range(7))
    model_data.at[index, 'val_accuracy'] = report['accuracy']
    model_data.at[index,'val_precision'] = report['macro avg']['precision']
    model_data.at[index,'val_recall'] = report['macro avg']['recall']


model_small_without_hand
model_small_with_hand
model_big_without_hand
model_big_with_hand


In [7]:
precision

array([    0.93699,      0.9306,     0.98953,     0.97037,     0.98765,     0.99439,     0.98375])

In [8]:
model_data['inference_time'] = model_data['inference_time']*1000 #(ms)
model_data['inference_time'] = model_data['inference_time'].round(decimals=2)
model_data['accuracy'] = model_data['accuracy'].round(decimals =4)
model_data['val_accuracy'] = model_data['val_accuracy'].round(decimals =4)
model_data['precision'] = model_data['precision'].round(decimals =4)
model_data['val_precision'] = model_data['val_precision'].round(decimals =4)
model_data['recall'] = model_data['recall'].round(decimals =4)
model_data['val_recall'] = model_data['val_recall'].round(decimals =4)

In [9]:
model_data.head()

,Unnamed: 0,name,hand,unknown,size,...,inference_time,precision,recall,val_precision,val_recall
0,0,model_small_without_hand,False,False,small,...,31.29,0.9673,0.9689,0.9200,0.9250
1,1,model_small_with_hand,True,False,small,...,31.29,0.9628,0.9643,0.9242,0.9272
2,2,model_big_without_hand,False,False,big,...,31.29,0.9711,0.9730,0.9305,0.9317
3,3,model_big_with_hand,True,False,big,...,31.28,0.9705,0.9742,0.9218,0.9300


In [10]:
model_data.to_csv('results/classifier_results2.csv',index=False)